In [1]:
import pandas as pd

In [2]:
import os

In [29]:
import numpy as np

In [3]:
path = os.path.join("Resources", "purchase_data.csv")

In [4]:
df = pd.read_csv(path)

In [5]:
df = df

In [6]:
#display the total count of players

In [7]:
df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [8]:
pd.DataFrame ([df["SN"].nunique()],columns=['Total Players'])

,Total Players
0,576


In [9]:
summary = [df["Item ID"].nunique(), df["Price"].mean(), df["Purchase ID"].count(), df["Price"].sum()]

In [10]:
pd.DataFrame([summary], columns=["Number of Unique Items", "Average Price", "Number of Purchases", "Total Revenue"])

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,3.050987,780,2379.77


In [43]:
df.groupby(["Gender"]).size().reset_index(name="Total Count")

,Gender,Total Count
0,Female,113
1,Male,652
2,Other / Non-Disclosed,15
